In [ ]:
!pip install snowflake

In [ ]:
!pip install snowflake-ml-python

In [ ]:
!pip install snowflake-snowpark-python

# Connect to Snowflake

In [4]:
import os

In [5]:
from google.colab import userdata  # for Google Colab authentication
from snowflake.snowpark.session import Session

connection_params = {
    "account": userdata.get('SNOWFLAKE_ACCOUNT'),
    "user": userdata.get('SNOWFLAKE_USER'),
    "password": userdata.get('SNOWFLAKE_USER_PASSWORD'),
    "role": userdata.get('SNOWFLAKE_ROLE'),
    "database": userdata.get('SNOWFLAKE_DATABASE'),
    "schema": userdata.get('SNOWFLAKE_SCHEMA'),
    "warehouse": userdata.get('SNOWFLAKE_WAREHOUSE')
}
snowpark_session = Session.builder.configs(connection_params).create()

In [6]:
snowpark_session

Test CORTEX LLM

In [7]:
from snowflake.cortex import complete

print(complete("mistral-large2", "how do snowflakes get their unique patterns?"))

 Snowflakes get their unique patterns through a complex process of crystallization that occurs as water vapor freezes in the atmosphere. Here's a simplified breakdown of how this happens:

1. **Nucleation**: The process begins with a tiny particle in the air, such as a speck of dust or pollen, which acts as a nucleus. Water vapor condenses around this nucleus and starts to freeze.

2. **Crystal Growth**: As more water vapor freezes onto the initial ice crystal, it forms a hexagonal prism shape. This is because the water molecules arrange themselves in a hexagonal pattern due to the hydrogen bonds between them.

3. **Branching**: As the ice crystal grows, it can develop branches, or "arms." The exact pattern of these branches is influenced by the temperature and humidity conditions in the cloud. Different conditions lead to different growth rates and patterns.

4. **Complex Patterns**: As the snowflake falls through the cloud, it encounters varying temperatures and humidity levels. Thes

In [8]:
import pandas as pd

df = pd.read_csv("hf://datasets/ai-4-all/sklearn-ml/sklearn_ml.csv")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
df

,input,output,instruction
0,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.linear_mod...,Write functional matchine learning classifcati...
1,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.neighbors ...,Write functional matchine learning classifcati...
2,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.tree impor...,Write functional matchine learning classifcati...
3,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.svm import...,Write functional matchine learning classifcati...
4,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.ensemble i...,Write functional matchine learning classifcati...
...,...,...,...
16813,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.neighbors ...,Write functional matchine learning regression ...
16814,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.gaussian_p...,Write functional matchine learning regression ...
16815,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.tree impor...,Write functional matchine learning regression ...
16816,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.neural_net...,Write functional matchine learning regression ...


In [10]:
df['instruction'].value_counts()

,count
instruction,
Write functional matchine learning regression code using the following instuctions and information,10812
Write functional matchine learning classifcation code using the following instuctions and information,6006


In [11]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['instruction'], random_state=42)

print("Train DataFrame:")
train_df
print("\nTest DataFrame:")
test_df

Train DataFrame:

Test DataFrame:


,input,output,instruction
3389,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.neighbors ...,Write functional matchine learning classifcati...
8170,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.linear_mod...,Write functional matchine learning regression ...
9191,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.kernel_rid...,Write functional matchine learning regression ...
9256,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.neural_net...,Write functional matchine learning regression ...
15170,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.gaussian_p...,Write functional matchine learning regression ...
...,...,...,...
5503,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.neighbors ...,Write functional matchine learning classifcati...
1267,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.gaussian_p...,Write functional matchine learning classifcati...
13305,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.linear_mod...,Write functional matchine learning regression ...
11824,Write python functions to make a machine learn...,\nimport pandas as pd\nfrom sklearn.neural_net...,Write functional matchine learning regression ...


In [12]:
print("Train set shape:", train_df.shape)
print("Test set shape:", test_df.shape)

Train set shape: (13454, 3)
Test set shape: (3364, 3)


In [19]:
import snowflake.connector
from tqdm.auto import tqdm
from typing import List, Dict

class BatchSklearnProcessor:
    def __init__(self, connection_params: Dict, batch_size: int = 1000):
        """Initialize the processor with connection params and batch size"""
        self.connection_params = connection_params
        self.batch_size = batch_size

    def batch_insert_to_snowflake(self, cursor, snowflake_connector: snowflake.connector.SnowflakeConnection,
                                 records: List[Dict]):
        """Batch insert records into Snowflake"""
        insert_query = """
        INSERT INTO sklearn_query_store (
            input,
            output,
            instruction
        ) VALUES (%s, %s, %s)
        """

        batch_data = []
        total_records = len(records)

        with tqdm(total=total_records, desc="Storing in Snowflake") as pbar:
            for record in records:
                batch_data.append((
                    record['input'],
                    record['output'],
                    record['instruction']
                ))

                if len(batch_data) >= self.batch_size or (pbar.n + 1) == total_records:
                    try:
                        cursor.executemany(insert_query, batch_data)
                        snowflake_connector.commit()
                        pbar.update(len(batch_data))
                        batch_data = []
                    except Exception as e:
                        print(f"Error inserting batch: {str(e)}")
                        snowflake_connector.rollback()
                        raise e

    def process_and_store(self, df):
        """Process DataFrame and store in Snowflake using batch operations"""
        print("Processing data...")

        records = []
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Preparing data"):
            record = {
                'input': row['input'],
                'output': row['output'],
                'instruction': row['instruction']
            }
            records.append(record)

        snowflake_connector = snowflake.connector.connect(**self.connection_params)
        cursor = snowflake_connector.cursor()

        try:
            print("Creating/checking table schema...")
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS sklearn_query_store (
                input TEXT,
                output TEXT,
                instruction TEXT)
            """)

            self.batch_insert_to_snowflake(cursor, snowflake_connector, records)

            print("Creating Cortex Search service...")
            cursor.execute("""
            CREATE OR REPLACE CORTEX SEARCH SERVICE sklearn_code_search_svc
            ON search_text
            WAREHOUSE = COMPUTE_WH
            TARGET_LAG = '1 hour'
            AS
                SELECT
                    input,
                    output,
                    instruction,
                    ('Instruction\n\n' || instruction || '\n\n\ Input Prompt\n\n' || input || '\n\n\ Output code (sklearn)\n\n' || output) as search_text
                FROM sklearn_query_store
            """)

        except Exception as e:
            print(f"Error during processing: {str(e)}")
            raise e
        finally:
            snowflake_connector.close()

In [20]:
processor = BatchSklearnProcessor(connection_params)
processor.process_and_store(train_df)

Processing data...


Preparing data:   0%|          | 0/13454 [00:00<?, ?it/s]

Creating/checking table schema...


Storing in Snowflake:   0%|          | 0/13454 [00:00<?, ?it/s]

Creating Cortex Search service...


In [21]:
from snowflake.core import Root
from snowflake.snowpark.session import Session
from typing import List, Dict

class SklearnCodeRetriever:
    def __init__(self, session: Session, limit_to_retrieve: int = 4):
        """Initialize the retriever

        Args:
            session: Snowflake session
            limit_to_retrieve: Maximum number of results to return
        """
        self._session = session
        self._limit_to_retrieve = limit_to_retrieve

    def retrieve(self, query: str) -> List[Dict]:
        """Retrieve relevant Python code based on natural language query

        Args:
            query: Natural language query to search for

        Returns:
            List of dictionaries containing matched Python code and prompts
        """
        root = Root(self._session)
        cortex_search_service = (
            root
            .databases[self._session.get_current_database()]
            .schemas[self._session.get_current_schema()]
            .cortex_search_services['sklearn_code_search_svc']
        )

        response = cortex_search_service.search(
            query=query,
            columns=[
                "input",
                "output",
                "instruction",
                "search_text"
            ],
            limit=self._limit_to_retrieve
        )

        if response.results:
            return [
                {
                    'input': result['input'],
                    'output': result['output'],
                    'instruction' : result['instruction'],
                    'search_text': result.get('search_text', '')
                }
                for result in response.results
            ]
        return []

    def format_results(self, results: List[Dict]) -> str:
        """Format results for display

        Args:
            results: List of search results

        Returns:
            Formatted string of results
        """
        if not results:
            return "No matching Python code found."

        formatted_output = []
        for i, result in enumerate(results, 1):
            formatted_output.append(f"\nResult {i}:")
            formatted_output.append(f"Input: {result['input']}")
            formatted_output.append("Output Code:")
            formatted_output.append("```python")
            formatted_output.append(result['output'])
            formatted_output.append("```")
            formatted_output.append(f"Instruction: {result['instruction']}")
            formatted_output.append("-" * 80)

        return "\n".join(formatted_output)

In [22]:
retriever = SklearnCodeRetriever(snowpark_session)

results = retriever.retrieve("how to make binary classification using sklearn")

print(retriever.format_results(results))


Result 1:
Input: Write python functions to make a machine learning BNB classification model based on a dataset named processed-data.csv having columns ['Tiredness', 'Dry-Cough', 'Difficulty-in-Breathing', 'Sore-Throat', 'None_Sympton', 'Pains', 'Nasal-Congestion', 'Runny-Nose', 'None_Experiencing', 'Age_0-9', 'Age_10-19', 'Age_20-24', 'Age_25-59', 'Age_60+', 'Gender_Female', 'Gender_Male', 'Severity_Mild', 'Severity_Moderate', 'Severity_None'] with the target variable being None_Experiencing having the classification evaluation metrics accuracy, precision, recall, f1 score
Output Code:
```python

import pandas as pd
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

df = pd.read_csv('./classification/classification_data/processed-data.csv')
target_variable = 'None_Experiencing'

def BNBClassificationModel(df, target_variable):
    X = df.drop

# Evaluation

In [23]:
from snowflake.core import Root
from snowflake.snowpark.session import Session
from typing import List, Dict
from tqdm.auto import tqdm

class RAGSklearnGenerator:
    def __init__(self, session: Session, model_name: str = "mistral-large2", num_examples: int = 3):
        """Initialize the RAG Python generator

        Args:
            session: Snowflake session
            model_name: Name of the Cortex model to use
            num_examples: Number of examples to retrieve
        """
        self.session = session
        self.root = Root(session)
        self.model_name = model_name
        self.num_examples = num_examples

    def retrieve_examples(self, query: str) -> List[Dict]:
        """Retrieve similar examples from Cortex Search

        Args:
            query: Natural language query
        """
        search_service = (
            self.root
            .databases[self.session.get_current_database()]
            .schemas[self.session.get_current_schema()]
            .cortex_search_services['sklearn_code_search_svc']
        )

        response = search_service.search(
            query=query,
            columns=["input", "output", "instruction"],
            limit=self.num_examples
        )

        return response.results if response.results else []

    def create_prompt(self, question: str, examples: List[Dict]) -> str:
        """Create prompt for Python code generation

        Args:
            question: Natural language question
            examples: Similar examples for few-shot learning
        """
        prompt = f"[INST]\nAs an expert Python programmer, generate sklearn implementation code for the following task:\n\n"
        prompt += f"Task: {question}\n\n"
        prompt += "Here are some similar examples to help guide you:\n\n"

        for i, example in enumerate(examples, 1):
            prompt += f"Example {i}:\n"
            prompt += f"Instruction: {example['instruction']}\n"
            prompt += f"Prompt: {example['input']}\n"
            prompt += "Python Code (sklearn):\n"
            prompt += "```python\n"
            prompt += f"{example['output']}\n"
            prompt += "```\n\n"

        prompt += "Based on these examples, generate Python code using sklearn for the original task:\n"
        prompt += f"{question}\n\n"
        prompt += "Output only the Python code without any explanation or additional text.\n"
        prompt += "[/INST]"

        return prompt

    def run_cortex_complete(self, prompt: str) -> str:
        """Run Cortex Complete model

        Args:
            prompt: Input prompt for the model
        """
        result = self.session.sql(
            "SELECT snowflake.cortex.complete(?, ?)",
            params=[self.model_name, prompt]
        ).collect()[0][0]
        return result

    def generate_python(self, question: str) -> Dict:
        """Generate Python code for a given question

        Args:
            question: Natural language question
        """
        # Retrieve similar examples
        examples = self.retrieve_examples(question)

        # Create prompt with examples
        prompt = self.create_prompt(question, examples)

        # Generate Python code using Cortex Complete
        generated_code = self.run_cortex_complete(prompt).strip()

        # Clean up the generated code if it contains markdown markers
        if generated_code.startswith("```python"):
            generated_code = generated_code.replace("```python", "").replace("```", "").strip()

        return {
            'question': question,
            'generated_code': generated_code,
            'examples_used': examples,
            'prompt_used': prompt
        }

In [25]:
from typing import List, Dict
from difflib import SequenceMatcher
from tqdm.auto import tqdm

class SklearnGenerationEvaluator:
    def __init__(self, generator: RAGSklearnGenerator, test_df: pd.DataFrame):
        """Initialize evaluator

        Args:
            generator: RAGPythonGenerator instance
            test_df: Test dataset DataFrame with prompt and python columns
        """
        self.generator = generator
        self.test_df = test_df

    def normalize_python(self, code: str) -> str:
        """Normalize Python code for comparison

        Args:
            code: Python code string to normalize
        """
        if not code:
            return ""

        code = code.replace("```python", "").replace("```", "")

        lines = [
            line.strip()
            for line in code.split('\n')
            if line.strip() and not line.strip().startswith('#')
        ]

        return ' '.join(lines)

    def calculate_similarity(self, code1: str, code2: str) -> float:
        """Calculate similarity between two Python code snippets"""
        norm_code1 = self.normalize_python(code1)
        norm_code2 = self.normalize_python(code2)
        return SequenceMatcher(None, norm_code1, norm_code2).ratio()

    def evaluate_single_query(self, row: pd.Series) -> Dict:
        """Evaluate a single code generation"""
        try:
            result = self.generator.generate_python(question=row['input'])

            similarity = self.calculate_similarity(
                result['generated_code'],
                row['output']
            )

            return {
                'input': row['input'],
                'true_code': row['output'],
                'generated_code': result['generated_code'],
                'similarity_score': similarity,
                'exact_match': similarity > 0.95,
                'num_examples': len(result['examples_used']),
                'prompt_used': result['prompt_used']
            }
        except Exception as e:
            print(f"Error evaluating code: {str(e)}")
            return None

    def evaluate_batch(self, sample_size: int = None) -> Dict:
        """Evaluate generator on test set"""
        evaluation_df = self.test_df
        if sample_size:
            evaluation_df = self.test_df.sample(n=sample_size, random_state=42)

        results = []
        for idx, row in tqdm(evaluation_df.iterrows(), total=len(evaluation_df), desc="Evaluating queries"):
            result = self.evaluate_single_query(row)
            if result:
                results.append(result)

        metrics = {
            'total_queries': len(results),
            'exact_matches': sum(r['exact_match'] for r in results),
            'average_similarity': sum(r['similarity_score'] for r in results) / len(results) if results else 0,
            'results': results
        }

        return metrics

    def print_evaluation_report(self, metrics: Dict):
        """Print detailed evaluation report"""
        print("\n=== Python Code Generation Evaluation Report ===")
        print(f"Total Tasks Evaluated: {metrics['total_queries']}")
        print(f"Exact Matches: {metrics['exact_matches']}")
        print(f"Exact Match Rate: {metrics['exact_matches']/metrics['total_queries']:.2%}")
        print(f"Average Similarity Score: {metrics['average_similarity']:.3f}")

        results_df = pd.DataFrame(metrics['results'])

        print("\nTop 3 Best Generations:")
        for _, row in results_df.nlargest(3, 'similarity_score').iterrows():
            print(f"\nPrompt: {row['input']}")
            print(f"Similarity Score: {row['similarity_score']:.3f}")
            print("\nGenerated Python Code:")
            print("```python")
            print(row['generated_code'])
            print("```")
            print("\nTrue Python Code:")
            print("```python")
            print(row['true_code'])
            print("```")
            print("-" * 80)

In [ ]:
print("Initializing generator and evaluator...")
generator = RAGSklearnGenerator(
    session=snowpark_session,
    model_name="mistral-large2",
    num_examples=3
)

evaluator = SklearnGenerationEvaluator(
    generator=generator,
    test_df=test_df
)

print("Running evaluation...")
metrics = evaluator.evaluate_batch(sample_size=100)

evaluator.print_evaluation_report(metrics)

Initializing generator and evaluator...
Running evaluation...


Evaluating queries:   0%|          | 0/100 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




=== Python Code Generation Evaluation Report ===
Total Tasks Evaluated: 100
Exact Matches: 80
Exact Match Rate: 80.00%
Average Similarity Score: 0.896

Top 3 Best Generations:
KeyError: 'prompt'

The above exception was the direct cause of the following exception:

KeyError: 'prompt'

During handling of the above exception, another exception occurred:

AttributeError: 'KeyError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

AssertionError
KeyError: 'prompt'

The above exception was the direct cause of the following exception:

KeyError: 'prompt'

During handling of the above exception, another exception occurred:

AttributeError: 'KeyError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

TypeError: object of type 'NoneType' has no len()

During handling of the above exception, another exception occurred:

AttributeError: 'TypeError' object has no attr

Average Similarity Score: 0.896
